# Dense Embedding 을 활용한 ODQA 시스템 만들기

### Requirements

In [ ]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1

In [ ]:
import torch
import random
from pprint import pprint

## Dense Embedding 을 활용한 Open-domain Question Answering 시스템 만들기


5강에서 배운대로 dense embedding을 만드는 encoder 을 학습시키기

또는 학습된 encoder 파일을 가져와서 진행하기

In [ ]:
# 1. 미리 학습해둔 encoder file 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx" -O encoders.tar.gz && rm -rf /tmp/cookies.txt
# 2. .tar.gz file 압축해제 
!tar -xf ./encoders.tar.gz

--2021-08-13 06:43:14--  https://docs.google.com/uc?export=download&confirm=&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx
Resolving docs.google.com (docs.google.com)... 142.251.2.102, 142.251.2.113, 142.251.2.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘encoders.tar.gz’

encoders.tar.gz         [ <=>                ]   3.05K  --.-KB/s    in 0s      

2021-08-13 06:43:14 (48.8 MB/s) - ‘encoders.tar.gz’ saved [3126]


gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [ ]:
!du -sh encoders.tar.gz

4.0K	encoders.tar.gz


In [ ]:
# google drive 에 올려둔 미리 학습해둔 인코더 불러오기
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

# TODO: load model from path
p_encoder = ...
q_encoder = ...

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Dense retriever 가져오기

In [ ]:
# klue data가 포함된 wiki documents 가져오기
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

--2021-08-10 02:44:29--  https://docs.google.com/uc?export=download&confirm=Mp9O&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 142.250.101.102, 142.250.101.100, 142.250.101.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.101.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-5k-docs.googleusercontent.com/docs/securesc/p0jim3rrvtn2adr8mj523glldqg5b2du/ddc4h9746e5ab1iag5h27lumsk9c66sr/1628563425000/10000090644219567406/04489312442488276622Z/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download [following]
--2021-08-10 02:44:29--  https://doc-04-5k-docs.googleusercontent.com/docs/securesc/p0jim3rrvtn2adr8mj523glldqg5b2du/ddc4h9746e5ab1iag5h27lumsk9c66sr/1628563425000/10000090644219567406/04489312442488276622Z/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download
Resolving doc-04-5k-docs.googleusercontent.com (doc-04-5k-docs.googleusercontent.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connec

In [ ]:
# First load wikipedia dump
import json

dump_path = 'wikipedia_documents.json' 
with open(dump_path, 'r') as f:
    wiki = json.load(f)

corpus = [document['text'] for document_id, document in wiki.items()]

사전에 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [ ]:
from tqdm.auto import tqdm

# takes about 40mins ~ 1hr
# you can use below cell to download embedding
with torch.no_grad():
    p_encoder.eval()
    p_embs = []
    for p in tqdm(corpus):
        p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

In [ ]:
# download wiki_p_embs
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI" -O wiki_p_embs.pt && rm -rf /tmp/cookies.txt

p_embs = torch.load("wiki_p_embs.pt")

retrieval 과정을 함수화 하기

In [ ]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [ ]:
def get_relevant_doc(p_encoder, q_encoder, query, k=1):
    with torch.no_grad():
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

    # TODO: get dot product scores by using torch.matmul
    dot_prod_scores = ...

    # TODO: obtain rank (use torch.argsort)
    rank = ...
    
    return dot_prod_scores.squeeze(), rank[:k]

In [ ]:
""" 원하는 질문을 입력해보기 """
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
_, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)

""" 결과 확인 """

print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
print(corpus[doc_id.item()])

Enter any question: 대한민국의 대통령은 누구인가?
******************** Result ********************
[Search query]
 대한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 41846
대한민국에서 1979년에서 이듬해 1980년까지 최규하(崔圭夏) 대통령에 이어서 박충훈(朴忠勳) 국무총리 서리 겸 대통령 권한대행이 재임을 하였고 전두환(全斗煥) 육군 보안사령관 겸 중앙정보부 부장 서리가 실권을 하였던 시기의 절정으로 접어든 1980년 5월 31일에서 같은 해 1980년 10월 27일까지 국가보위비상대책위원회(國家保衛非常對策委員會)의 위원장(委員長)은 국보위 대표위원장(國保委 代表委員長)까지를 포함하여 모두 4명이 거쳐갔고 1980년 10월 27일에 남덕우(南悳祐) 위원장 서리를 끝으로 국가보위비상대책위원회 위원장 직위 체제가 전격 종결되었다.


In [ ]:
""" 상위 5개를 추출하여 점수 확인 """
dot_prod_scores, rank = get_relevant_doc(p_encoder, q_encoder, query, k=5)

for i in range(5):
    print(rank[i])
    print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
    print(corpus[rank[i]])

tensor(41846)
Top-1 passage with score 38.0327
대한민국에서 1979년에서 이듬해 1980년까지 최규하(崔圭夏) 대통령에 이어서 박충훈(朴忠勳) 국무총리 서리 겸 대통령 권한대행이 재임을 하였고 전두환(全斗煥) 육군 보안사령관 겸 중앙정보부 부장 서리가 실권을 하였던 시기의 절정으로 접어든 1980년 5월 31일에서 같은 해 1980년 10월 27일까지 국가보위비상대책위원회(國家保衛非常對策委員會)의 위원장(委員長)은 국보위 대표위원장(國保委 代表委員長)까지를 포함하여 모두 4명이 거쳐갔고 1980년 10월 27일에 남덕우(南悳祐) 위원장 서리를 끝으로 국가보위비상대책위원회 위원장 직위 체제가 전격 종결되었다.
tensor(6978)
Top-2 passage with score 37.7841
인물평으로는 총명하고 지성적이라는 평가가 있다. 1948년 대한민국의 건국과 수호에 발군의 역할을 한 외교관이 1948년 9월 대한민국 정부 수립 직후 UN에 대한민국 정부 승인을 성사시킨 것과 1950년 6월부터 한국 전쟁 당시 미국과 국제 사회를 설득하여 미군과 UN군의 한국 전쟁 참전을 이끌어낸 점이 높이 평가된다. 단독정부 수립이후 남한에 대한 국제사회의 승인을 이끌어냈고, 6·25 전쟁 시 미군과 유엔군의 파병을 이끌어내 남한의 자유민주주의 체제를 지키는 데 있어 결정적인 역할을 한 건국공로자였다. 대한민국 정부 수립 후 최초로 의원내각제를 실시하였다는 평가와, 정부수립 후 최초로 민주적 선거절차에 의한 정권교체라는 의미를 둘 수 있다. 박정희 정권은 5·16 군사정변 이후 장면을 부정, 부패 무능의 상징이라 주장하였으나 박정희는 장면 내각 출범 이후 19일 뒤부터 군사 정변을 모의하였다. 군사 정권은 장면의 통솔력 부족과 무능력함을 집중적으로 조명했으나 이는 장면에게 부여된 시간적 여유가 없었다는 반론로 제기되고 있다.

한국헌법사를 쓴 김영수에 의하면 “급변하는 정치적 현실 속에서 이익의 조직화를 이루어 내는 능력을 갖고 있던

## 훈련된 MRC 모델 가져오기


In [ ]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [ ]:
model_name = 'sangrimlee/bert-base-multilingual-cased-korquad'
mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

In [ ]:
mrc_model = mrc_model.eval()

In [ ]:
def get_answer_from_context(context, question, model, tokenizer):
    # TODO: get encoded dictionary from question - use tokenizer.encode_plus
    encoded_dict = ...
    
    non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)
    inputs = {
    'input_ids': torch.tensor([encoded_dict['input_ids']], dtype=torch.long),
    'attention_mask': torch.tensor([encoded_dict['attention_mask']], dtype=torch.long),
    'token_type_ids': torch.tensor([encoded_dict['token_type_ids']], dtype=torch.long)
    }

    outputs = model(**inputs)
    start, end = torch.max(outputs.start_logits, axis=1).indices.item(), torch.max(outputs.end_logits, axis=1).indices.item()
    if start == 0 and end == 0:
        answer = "This is not answerable"
    else:
        answer = tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [ ]:
context = corpus[doc_id.item()]
answer = get_answer_from_context(context, query, mrc_model, tokenizer)
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
pprint(corpus[doc_id.item()], compact=True)
print(f"[Answer Prediction from the model]: {answer}")

******************** Result ********************
[Search query]
 대한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 41846
('대한민국에서 1979년에서 이듬해 1980년까지 최규하(崔圭夏) 대통령에 이어서 박충훈(朴忠勳) 국무총리 서리 겸 대통령 권한대행이 '
 '재임을 하였고 전두환(全斗煥) 육군 보안사령관 겸 중앙정보부 부장 서리가 실권을 하였던 시기의 절정으로 접어든 1980년 5월 31일에서 '
 '같은 해 1980년 10월 27일까지 국가보위비상대책위원회(國家保衛非常對策委員會)의 위원장(委員長)은 국보위 대표위원장(國保委 '
 '代表委員長)까지를 포함하여 모두 4명이 거쳐갔고 1980년 10월 27일에 남덕우(南悳祐) 위원장 서리를 끝으로 국가보위비상대책위원회 '
 '위원장 직위 체제가 전격 종결되었다.')
[Answer Prediction from the model]: 최규하


## 통합해서 ODQA 시스템 구축

In [ ]:
def open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1):
    # 1. Retrieve k relevant docs by usign sparse matrix
    _, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)
    context = corpus[doc_id.item()]

    # 2. Predict answer from given doc by using MRC model
    answer = get_answer_from_context(context, query, mrc_model, tokenizer)
    print("{} {} {}".format('*'*20, 'Result','*'*20))
    print("[Search query]\n", query, "\n")
    print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
    pprint(corpus[doc_id.item()], compact=True)
    print(f"[Answer Prediction from the model]: {answer}")

In [ ]:
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1)

Enter any question: 대한민국의 대통령은 누구인가?
******************** Result ********************
[Search query]
 대한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 41846
('대한민국에서 1979년에서 이듬해 1980년까지 최규하(崔圭夏) 대통령에 이어서 박충훈(朴忠勳) 국무총리 서리 겸 대통령 권한대행이 '
 '재임을 하였고 전두환(全斗煥) 육군 보안사령관 겸 중앙정보부 부장 서리가 실권을 하였던 시기의 절정으로 접어든 1980년 5월 31일에서 '
 '같은 해 1980년 10월 27일까지 국가보위비상대책위원회(國家保衛非常對策委員會)의 위원장(委員長)은 국보위 대표위원장(國保委 '
 '代表委員長)까지를 포함하여 모두 4명이 거쳐갔고 1980년 10월 27일에 남덕우(南悳祐) 위원장 서리를 끝으로 국가보위비상대책위원회 '
 '위원장 직위 체제가 전격 종결되었다.')
[Answer Prediction from the model]: 최규하
